<a href="https://colab.research.google.com/github/himanshunaidu/cnn_image_caption/blob/master/ImageCaption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [1]:
from google.colab import files
uploaded = files.upload()

Saving load_dataset.py to load_dataset.py
Saving prep_ds.py to prep_ds.py
Saving preprocess_images.py to preprocess_images.py
Saving preprocess_labels.py to preprocess_labels.py


In [2]:
import os
import tensorflow as tf
import numpy as np
import math
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2, inception_v3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as kb
from tqdm import tqdm
import re

In [3]:
print(tf.__version__)

2.5.0


In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
!ls

gdrive		 prep_ds.py	       preprocess_labels.py  sample_data
load_dataset.py  preprocess_images.py  __pycache__


# LOCAL IMPORTS AND VARIABLES

In [6]:
base_path = './gdrive/MyDrive/ImageCaptions/'
caption_file_path = './Flickr8k.token.txt'
ds_path = 'Flicker8k_Dataset/'
feature_path = 'Flicker8k_Dataset_Features/'
filler_path = 'Filler/'
train_images_path = 'Flickr_8k.trainImages.txt'

# LOAD IMAGES AND CAPTIONS

In [7]:
from load_dataset import loadCaptions, getImages

In [8]:
img_list = getImages(os.path.join(base_path, train_images_path))

In [9]:
capdict = loadCaptions(os.path.join(base_path, caption_file_path))

In [10]:
capdictset = {}
for i, key in enumerate(capdict.keys()):
  if i>3: break
  capdictset[key] = capdict.get(key)

# PREPROCESS IMAGES

In [11]:
from preprocess_images import load_image, get_images, preprocess_images

In [12]:
img_name_list, img_path_list, img_dataset = get_images(base_path, train_images_path, ds_path)

6001
./gdrive/MyDrive/ImageCaptions/Flicker8k_Dataset/2513260012_03d33305cf.jpg


In [13]:
# USE THIS FUNCTION TO SAVE ALL THE PRE-PROCESSED IMAGE FEATURES AS GENERATED BY INCEPTIONV3
# preprocess_images(img_name_list, img_path_list, img_dataset, ds_path, feature_path)

# PREPROCESS CAPTIONS

In [14]:
# USE THE FUNCTIONS TO PRE-PROCESS ALL THE CAPTIONS

In [15]:
from preprocess_labels import captions_clean, add_token, subset_data_dict, all_captions, max_caption_length, create_tokenizer

In [16]:
captions_clean(capdict)

In [17]:
capdict[list(capdict.keys())[0]]
# list(capdict.keys())[0]

['child inpink dress is climbing upset of stairs in an entry way',
 'girl going intowooden building',
 'little girl climbing intowooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl inpink dress going intowooden cabin']

In [18]:
training_dict = subset_data_dict (capdict, img_name_list) #startseq and enseq are being added

In [19]:
tokenizer, vocab_size, max_caption_words = create_tokenizer(training_dict)

# PREP DATASET

In [20]:
from prep_ds import data_prep, map_func, lazy_load_ds

In [21]:
train_X, train_y = data_prep(training_dict, tokenizer, max_caption_words, vocab_size, base_path, ds_path)

In [22]:
print(train_X[0], train_y[0])

./gdrive/MyDrive/ImageCaptions/Flicker8k_Dataset/997338199_7343367d7f.jpg [   1   38   62  105  371 2026    2    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]


In [23]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000

In [24]:
dataset = lazy_load_ds(train_X, train_y, BUFFER_SIZE, BATCH_SIZE)

In [25]:
dataset

<PrefetchDataset shapes: (<unknown>, <unknown>), types: (tf.float32, tf.int32)>